In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet

# ✅ 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250422\C150_G0000_00013.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250422\2025년0422_전사 정지조건리스트_완성_서비스번호.xlsx'
output_csv_path = r'D:\시설\정지\일일정지\20250422\2025년0422_전사 정지조건리스트_완성_서비스번호.csv'

try:
    # ✅ 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ KTT월정료 통일: 'KTT월정료(조정)' → 'KTT월정료' 우선 적용
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''  # 없는 경우 빈 컬럼 생성

    # ✅ 날짜 포맷 정제 함수
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 월정료 구간 분류
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만 이하'
            elif amount <= 70000:
                return '5만 ~ 7만 이하'
            elif amount <= 100000:
                return '7만 ~ 10만 이하'
            elif amount <= 200000:
                return '10만 ~ 20만 이하'
            elif amount <= 500000:
                return '20만 ~ 50만 이하'
            else:
                return '50만 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장 (한글 호환용)
    df.to_csv(output_csv_path, index=False, encoding='cp949')

    print(f"✅ 최종 저장 완료! (행 수: {df.shape[0]})")

except Exception as e:
    print("❌ 오류 발생:", e)


    위 코드에서 추가로 계약번호 컬럼기준으로 중복되는건은 KTT월정료(조정) 컬럼 금액을 합계하여 하나만 남기고 중복제외처리

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet

# ✅ 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250422\C150_G0000_00013.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250422\2025년0422_전사 정지조건리스트_완성_서비스번호.xlsx'
output_csv_path = r'D:\시설\정지\일일정지\20250422\2025년0422_전사 정지조건리스트_완성_서비스번호.csv'

try:
    # ✅ 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ 계약번호 기준 중복 제거: KTT월정료(조정) 합산 후 하나만 남김
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        agg_dict = {col: 'first' for col in df.columns if col != 'KTT월정료(조정)'}
        agg_dict['KTT월정료(조정)'] = 'sum'
        df = df.groupby('계약번호', as_index=False).agg(agg_dict)
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 날짜 포맷 정제 함수
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 월정료 구간 분류
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만 이하'
            elif amount <= 70000:
                return '5만 ~ 7만 이하'
            elif amount <= 100000:
                return '7만 ~ 10만 이하'
            elif amount <= 200000:
                return '10만 ~ 20만 이하'
            elif amount <= 500000:
                return '20만 ~ 50만 이하'
            else:
                return '50만 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장 (한글 호환용)
    df.to_csv(output_csv_path, index=False, encoding='cp949')

    print(f"✅ 최종 저장 완료! (행 수: {df.shape[0]})")

except Exception as e:
    print("❌ 오류 발생:", e)

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet

# ✅ 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250422\C150_G0000_00013.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250422\2025년0422_전사 정지조건리스트_완성_서비스번호.xlsx'
output_csv_path = r'D:\시설\정지\일일정지\20250422\2025년0422_전사 정지조건리스트_완성_서비스번호.csv'

try:
    # ✅ 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ 계약번호 기준 중복 제거 및 KTT월정료(조정) 합산
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        agg_dict = {col: 'first' for col in df.columns if col != 'KTT월정료(조정)'}
        agg_dict['KTT월정료(조정)'] = 'sum'
        df = df.groupby('계약번호', as_index=False).agg(agg_dict)
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 날짜 정제
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 월정료 구간
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만 이하'
            elif amount <= 70000:
                return '5만 ~ 7만 이하'
            elif amount <= 100000:
                return '7만 ~ 10만 이하'
            elif amount <= 200000:
                return '10만 ~ 20만 이하'
            elif amount <= 500000:
                return '20만 ~ 50만 이하'
            else:
                return '50만 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장 (UTF-8 with BOM — Excel 한글 호환)
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

    print(f"✅ 최종 저장 완료! (행 수: {df.shape[0]})")

except Exception as e:
    print("❌ 오류 발생:", e)

In [ ]:
import os
import shutil

# ✅ 1. 조회할 폴더 경로 설정
search_dir = r"D:\시설"  # 검색 시작 경로 (필요에 따라 수정)
target_dir = r"D:\시설\cs모음"  # .cs 파일을 모을 대상 폴더

# ✅ 2. 대상 폴더 없으면 생성
os.makedirs(target_dir, exist_ok=True)

# ✅ 3. 하위 폴더 포함하여 .cs 파일 검색 및 이동
moved_files = []
for root, dirs, files in os.walk(search_dir):
    for file in files:
        if file.lower().endswith(".cs"):
            src_path = os.path.join(root, file)
            dest_path = os.path.join(target_dir, file)

            # 같은 이름의 파일이 이미 존재하는 경우 번호 붙이기
            base, ext = os.path.splitext(file)
            counter = 1
            while os.path.exists(dest_path):
                dest_path = os.path.join(target_dir, f"{base}_{counter}{ext}")
                counter += 1

            shutil.move(src_path, dest_path)
            moved_files.append(dest_path)

# ✅ 4. 결과 출력
print(f"총 {len(moved_files)}개의 .cs 파일이 이동되었습니다.")
for f in moved_files:
    print(f"이동됨: {f}")

In [ ]:
from google.colab import drive
import shutil

# ✅ 드라이브 마운트
drive.mount('/content/drive')

# ✅ 로컬 파일에서 드라이브로 복사 (업로드 후에 사용)
shutil.copy('cs모음.zip', '/content/drive/MyDrive/시설/cs모음.zip')

print("드라이브에 복사 완료!")

In [ ]:
import shutil
import os

source_folder = r'D:\시설\cs모음'
zip_path = r'D:\시설\cs모음.zip'

# zip으로 압축
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', source_folder)

print(f"압축 완료: {zip_path}")

In [ ]:
from google.colab import files

# ✅ 파일 업로드 창 표시 (사용자가 zip 파일 선택)
uploaded = files.upload()

# ✅ 업로드된 파일 확인
for fn in uploaded.keys():
    print(f"업로드된 파일: {fn}")